### This IPYNB file explains about Hybrid search that is Dense matrix(semantic search) + sparse matrix(exact keyword search)

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

In [2]:
docs = [
    Document(page_content="LangChain helps build LLM applications."),
    Document(page_content="Pinecone is a vector database for semantic search."),
    Document(page_content="The Eiffel Tower is located in Paris."),
    Document(page_content="Amazon Bedrock allows developers to use multiple foundation models."),
    Document(page_content="FAISS is a fast library for similarity search and clustering."),
    Document(page_content="Retrieval-Augmented Generation improves accuracy by grounding LLMs in external data.")
]


In [3]:
### Dense retriver

embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
dense_vectorstore=FAISS.from_documents(docs,embedding_model)
dense_retriver=dense_vectorstore.as_retriever()




In [4]:
### Sparse retriver

sparse_retriver=BM25Retriever.from_documents(docs)
sparse_retriver.k=2

### combine both retrivers

hybrid_retriver=EnsembleRetriever(
    retrievers=[dense_retriver,sparse_retriver],
    weight=[0.8,0.2]
    )

In [5]:
hybrid_retriver

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DB42A94050>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001DB42A94440>, k=2)], weights=[0.5, 0.5])

In [6]:
query="what is Faiss"

result=hybrid_retriver.invoke(query)

In [7]:
result

[Document(id='d8257e2a-8489-4892-a54e-64ee87b6810e', metadata={}, page_content='FAISS is a fast library for similarity search and clustering.'),
 Document(metadata={}, page_content='Retrieval-Augmented Generation improves accuracy by grounding LLMs in external data.'),
 Document(id='1a371f96-c11b-4c04-86f6-6caf3d71a6e2', metadata={}, page_content='The Eiffel Tower is located in Paris.'),
 Document(id='cf85fe13-2d78-4154-b6e6-9ef8398b0eee', metadata={}, page_content='Amazon Bedrock allows developers to use multiple foundation models.'),
 Document(id='a6b03fe0-9f25-4e88-a95f-3d302646d38a', metadata={}, page_content='LangChain helps build LLM applications.')]

In [8]:
###RaG pipeline 

from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [9]:
prompt=PromptTemplate.from_template(
    """answer the below question on the context what i provided you.
    question: {input}
    context: {context}
    """
)

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()   # Reads the .env file automatically

api_key = os.getenv("OPENAI_API_KEY")




In [12]:
from langchain.chat_models import init_chat_model




# Initialize model
model_name = "gpt-3.5-turbo"

llm = init_chat_model(
    model=model_name,
    api_key=api_key,
)

resp = llm.invoke("Explain what Retrieval-Augmented Generation is in simple words.")
print(resp)


content='Retrieval-Augmented Generation is a technology that combines two different types of artificial intelligence: retrieval-based models and generative models. \n\nRetrieval-based models search for relevant information or data from a large database or knowledge base and use this information to generate responses. Generative models, on the other hand, generate responses based on a set of rules or patterns without needing to search for external information.\n\nWhen these two types of models are combined in Retrieval-Augmented Generation, the system can provide more accurate and relevant responses by both searching for information and generating responses from scratch. This technology is commonly used in chatbots, search engines, and other natural language processing applications to improve the quality of generated content.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 20, 'total_tokens': 161, 'completion_tokens_deta